### imports

In [15]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import pickle as pk
import mlflow
import xgboost as xgb
import pickle
import os
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.metrics import root_mean_squared_error
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

!python -V

Python 3.12.7


### configs

In [16]:
# configs
# set the tracking uri to the local sqlite db
mlflow.set_tracking_uri("sqlite:///mlflow.db")
# set the experiment name
mlflow.set_experiment("NYC-taxi-duration-prediction")
# access to the ui
#$ mlflow ui --backend-store-uri sqlite:///mlflow.db

### preprocessing
raw_data_path = "./TAXI_DATA_FOLDER"
dataset = "green"
dest_path = "./output"



### preprocessing

In [13]:
def dump_pickle(obj, filename: str):
    with open(filename, "wb") as f_out:
        return pickle.dump(obj, f_out)


def read_dataframe(filename: str):
    df = pd.read_parquet(filename)

    df['duration'] = df['lpep_dropoff_datetime'] - df['lpep_pickup_datetime']
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)
    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)

    return df


def preprocess(df: pd.DataFrame, dv: DictVectorizer, fit_dv: bool = False):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    dicts = df[categorical + numerical].to_dict(orient='records')
    if fit_dv:
        X = dv.fit_transform(dicts)
    else:
        X = dv.transform(dicts)
    return X, dv

In [14]:
# Load parquet files
df_train = read_dataframe(
    os.path.join(raw_data_path, f"{dataset}_tripdata_2023-01.parquet")
)
df_val = read_dataframe(
    os.path.join(raw_data_path, f"{dataset}_tripdata_2023-02.parquet")
)
df_test = read_dataframe(
    os.path.join(raw_data_path, f"{dataset}_tripdata_2023-03.parquet")
)

# Extract the target
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values
y_test = df_test[target].values

# Fit the DictVectorizer and preprocess data
dv = DictVectorizer()
X_train, dv = preprocess(df_train, dv, fit_dv=True)
X_val, _ = preprocess(df_val, dv, fit_dv=False)
X_test, _ = preprocess(df_test, dv, fit_dv=False)

# Create dest_path folder unless it already exists
os.makedirs(dest_path, exist_ok=True)

# Save DictVectorizer and datasets
dump_pickle(dv, os.path.join(dest_path, "dv.pkl"))
dump_pickle((X_train, y_train), os.path.join(dest_path, "train.pkl"))
dump_pickle((X_val, y_val), os.path.join(dest_path, "val.pkl"))
dump_pickle((X_test, y_test), os.path.join(dest_path, "test.pkl"))